In [1]:
import numpy as np
from numpy import std
from numpy import mean
import pandas as pd
from random import randint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

ds = pd.read_csv('TRNcod.xls', delimiter = "\t")

# Shuffle no dataset
ds = ds.sample(frac=1).reset_index(drop=True)

In [ ]:
# print([d for d in ds.columns])

In [11]:
# inadimplentes = pd.DataFrame(list(filter(lambda x: x == 1, ds['IND_BOM_1_2'])))

# Selecionando quem é inadimplente
inadimplentes = ds[ds['IND_BOM_1_2'] == 1]

# Selecionando quem é adimplente
adimplente = ds[ds['IND_BOM_1_2'] == 0]

global treino_ina
global teste_ina
global valid_ina
global treino_adi
global teste_adi
global valid_adi


# Dividindo datasets
treino_ina = inadimplentes[:int(len(inadimplentes)/2)]
teste_ina  = inadimplentes[int(len(inadimplentes)/2):int((len(inadimplentes)*3)/4)]
valid_ina  = inadimplentes[int((len(inadimplentes)*3)/4):]

treino_adi = adimplente[:int(len(adimplente)/2)]
teste_adi  = adimplente[int(len(adimplente)/2):int((len(adimplente)*3)/4)]
valid_adi  = adimplente[int((len(adimplente)*3)/4):]

# Equalizando tamanho de datasets treino e validação dos inadimplentes ao de adimplentes
treino_ina = treino_ina.loc[treino_ina.index.repeat(2)].drop('INDEX', axis=1)
treino_ina["COPIA"] = treino_ina.duplicated()
treino_ina.sort_values(by="COPIA", inplace=True, ignore_index=True)
treino_ina = treino_ina.iloc[ : ( len(treino_adi) - len(treino_ina) ), :  ]
treino_ina.drop(columns=["COPIA"], axis=1)

valid_ina  = valid_ina.loc[valid_ina.index.repeat(2)].drop('INDEX', axis=1)
valid_ina["COPIA"] = valid_ina.duplicated()
valid_ina.sort_values(by="COPIA", inplace=True, ignore_index=True)
valid_ina = valid_ina.iloc[ : ( len(valid_adi) - len(valid_ina) ), :  ]
valid_ina.drop(columns=["COPIA"], axis=1)
# Fim da equalização

print('Tamanhos de inadimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_ina.values), len(teste_ina.values), len(valid_ina.values)))
print('Tamanhos de adimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_adi.values), len(teste_adi.values), len(valid_adi.values)))
print('Colunas: {}'.format(len([d for d in ds.columns])))

Tamanhos de inadimplentes: 
Treino: 127549
Teste: 33524
Validação: 63775

Tamanhos de adimplentes: 
Treino: 127549
Teste: 63774
Validação: 63775

Colunas: 246


In [ ]:
# https://stackoverflow.com/questions/29294983/how-to-calculate-correlation-between-all-columns-and-remove-highly-correlated-on

# # Create correlation matrix
# corr_matrix = ds.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find features with correlation greater than 0.89
# to_drop = [column for column in upper.columns if any(upper[column] > 0.89)]

# # Drop features 
# # ds.drop(to_drop, axis=1, inplace=True)
# ds.columns


# Colunas mais correlacionadas
# l = []
# for c in ds.columns[:-1]:
#     l.append({c: ds[c].corr(ds['IND_BOM_1_1'])})
#     # if (ds[c].corr(ds['IND_BOM_1_1']) > 0.0):
#         # print( c, ds[c].corr(ds['IND_BOM_1_1']) )
# l.sort(key=lambda x: list(x.values())[0])
# print(l)

In [ ]:
# Passos a fazer
# Esquema de experimentação (passo a passo do que vamos testar)
# MLP & Random Forest

In [12]:
# https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/

from keras.layers import Dropout, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping

# MLP
model = Sequential()

# Dropout
layer = Dropout(randint(0,100)/100)

# Regularização
# ???

# ds_2 = ds
ds_2 = ds.iloc[ 0:30000, : ]

# Taxa de parada quando não mais evoluir

model.add(Dense(512,input_dim=245, activation='tanh'))
model.add(Dropout(0.6))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam')
trY = ds_2['IND_BOM_1_2']
trX = ds_2.drop('IND_BOM_1_2', axis=1)
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])

# Parada antecipada caso em 10 epochs ela deixe de melhorar
paradinha = EarlyStopping(monitor='accuracy', mode='max', patience=10)

history = model.fit(trX, trY, epochs=50, verbose=1, callbacks=[paradinha])


Epoch 1/50
938/938 [==============================] - 2s 1ms/step - loss: 5.1866 - accuracy: 0.6314
Epoch 2/50
938/938 [==============================] - 1s 1ms/step - loss: 5.2277 - accuracy: 0.6375
Epoch 3/50
938/938 [==============================] - 1s 1ms/step - loss: 5.2522 - accuracy: 0.6424
Epoch 4/50
938/938 [==============================] - 1s 1ms/step - loss: 5.1888 - accuracy: 0.6379
Epoch 5/50
938/938 [==============================] - 1s 1ms/step - loss: 5.1775 - accuracy: 0.6308
Epoch 6/50
938/938 [==============================] - 1s 1ms/step - loss: 5.1360 - accuracy: 0.6177
Epoch 7/50
938/938 [==============================] - 1s 1ms/step - loss: 5.1588 - accuracy: 0.6255
Epoch 8/50
938/938 [==============================] - 1s 991us/step - loss: 5.2004 - accuracy: 0.6336
Epoch 9/50
938/938 [==============================] - 1s 1ms/step - loss: 5.2032 - accuracy: 0.6401
Epoch 10/50
938/938 [==============================] - 1s 1ms/step - loss: 5.1713 - accuracy: 0.62

In [13]:
ds_3 = ds.iloc[ 50001:100001, :]
trY = ds_3['IND_BOM_1_2']
trX = ds_3.drop('IND_BOM_1_2', axis=1)
x, y = model.evaluate(trX, trY, verbose=1)

1563/1563 [==============================] - 1s 508us/step - loss: 5.2510 - accuracy: 0.6596


In [10]:
# Cuidado: usar esses parâmetros muito elevados, ou o default (100 estimadores e profundidade ilimitada) vai travar seu computador

# Parâmetros default:
# n_estimators=100, *,
# criterion="gini",
# max_depth=None,
# min_samples_split=2,
# min_samples_leaf=1,
# min_weight_fraction_leaf=0.,
# max_features="auto",
# max_leaf_nodes=None,
# min_impurity_decrease=0.,
# min_impurity_split=None,
# bootstrap=True,
# oob_score=False,
# n_jobs=None,
# random_state=None,
# verbose=0,
# warm_start=False,
# class_weight=None,
# ccp_alpha=0.0,
# max_samples=None

random_forest = RandomForestClassifier(n_estimators=5, max_depth=8)
n_scores = cross_val_score(random_forest, ds.drop('IND_BOM_1_2', axis=1), ds['IND_BOM_1_2'], scoring='accuracy', n_jobs=1, error_score='raise')

print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.888 (0.081)


In [9]:
# Parâmetros default:
# *,loss='deviance', 
# learning_rate=0.1, 
# n_estimators=100,
# subsample=1.0, 
# criterion='friedman_mse', 
# min_samples_split=2,
# min_samples_leaf=1, 
# min_weight_fraction_leaf=0.,
# max_depth=3, 
# min_impurity_decrease=0.,
# min_impurity_split=None, 
# random_state=None, 
# max_features=None, verbose=0,
# max_leaf_nodes=None, 
# warm_start=False,
# validation_fraction=0.1, 
# n_iter_no_change=None, 
# tol=1e-4,
# ccp_alpha=0.0

gradient_boost = GradientBoostingClassifier(n_estimators = 5, max_depth = 8)
n_scores = cross_val_score(gradient_boost, ds.drop('IND_BOM_1_2', axis=1), ds['IND_BOM_1_2'], scoring='accuracy', n_jobs=1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 1.000 (0.000)
